<a href="https://colab.research.google.com/github/FrancescaMusella/MLDL-Project/blob/main/federated_finalversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision.transforms as T
import torchvision.transforms.functional as F
from torch import nn
from sklearn.model_selection import StratifiedKFold
import random
import copy
import numpy as np
import matplotlib.pyplot as plt

from collections import defaultdict
from collections import Counter

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [ ]:
#transformation of CIFAR100
transform = T.Compose([
    T.Resize((32, 32)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
from torchvision.datasets import CIFAR100
train_val=CIFAR100(root='.data/', train=True, download=True, transform=transform)
test=CIFAR100(root='.data/', train=False, download=True, transform=transform)

In [ ]:
#train-validation-test split
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split

targets = train_val.targets

train_indices, val_indices = train_test_split(
    range(len(targets)),
    test_size=0.1,
    stratify=targets,
    random_state=42
)

val = torch.utils.data.Subset(train_val, val_indices)

val_loader= torch.utils.data.DataLoader(val, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=False)

In [ ]:
#IID dataset creation
K=100
skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=42)

client_indices_list = []
client_dataset=[]
client_loader=[]
client_loader_mask=[]

targets = [train_val.targets[i] for i in train_indices]
for _, client_idx in skf.split(train_indices, targets):
    client_indices_list.append([train_indices[i] for i in client_idx])
    client_dataset.append(torch.utils.data.Subset(train_val, client_indices_list[-1]))
    client_loader.append(torch.utils.data.DataLoader(client_dataset[-1], batch_size=32, shuffle=True))

    # Stratified split: take 15% of total client data maintaining class distribution
    num_samples_mask = int(0.15 * len(client_dataset[-1]))
    targets_im = [client_dataset[-1][i][1] for i in range(len(client_dataset[-1]))]
    small_client_indices, _ = train_test_split(
    range(len(client_dataset[-1])),
    train_size=0.223,   #One image per class
    stratify=targets_im,
    random_state=42)

    small_client = torch.utils.data.Subset(client_dataset[-1], small_client_indices)
    small_client_loader = torch.utils.data.DataLoader(small_client, batch_size=1, shuffle=True)
    client_loader_mask.append(small_client_loader)

In [ ]:
#ViT-S/16
!git clone https://github.com/facebookresearch/dino.git
!ls

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
vits16_original = torch.hub.load('facebookresearch/dino:main', 'dino_vits16', pretrained=True).to(device)
vits16_new=copy.deepcopy(vits16_original)
print(vits16_new)

In [ ]:
#Change of the head and freezing of layers
vits16_new.head = torch.nn.Linear(in_features=384,
                    out_features=100,
                    bias=True).to(device)

for name, param in vits16_new.named_parameters():
    if "head" not in name and "patch_embed" not in name and 'proj' not in name and 'pos_drop' not in name and 'attn' not in name:
        param.requires_grad = False
    else:
        param.requires_grad = True

vits16=copy.deepcopy(vits16_new)

In [ ]:
def FedAvg(model, K, C, J, T, lr, momentum, weight_decay, loss_fn, client_loader, trainable_part, implementation,  client_loader_mask, test_loader, test):
  global_params = model.state_dict()

  val_acc_fed_list=[]
  val_loss_fed_list=[]

  for t in range(T):
      client_samples=random.sample(range(K), int(max(1, K*C)))
      local_params = []
      client_num_samples = []

      for client in client_samples:
          model_client=copy.deepcopy(model)
          num_samples = len(client_loader[client].dataset)
          client_num_samples.append(num_samples)

          if trainable_part == 'head':
              optimizer = torch.optim.SGD(model_client.head.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

          else:
              optimizer = torch.optim.SGD(model_client.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

          if implementation=='self':
                    sparsity=[0.1, 0.2, 0.3, 0.4, 0.66]
                    mask_fed=compute_fisher_mask(model_client,  client_loader_mask[client], sparsity, loss_fn)

          for loc_step in range(J):
              batch_idx=torch.randint(0, len(client_loader[client]), (1,)).item()
              for i, (inputs, labels) in enumerate(client_loader[client]):
                  if i == batch_idx:
                      inputs_client=inputs
                      labels_client=labels
                      break

              if implementation=='self':
                    train_sgd_sparse(loc_step, model_client, inputs_client, labels_client, loss_fn, lr, momentum, weight_decay, mask_fed)

              else:
                    train(loc_step, model_client, inputs_client, labels_client, loss_fn, optimizer,J,client)

          local_params.append(model_client.state_dict())

      total_samples = sum(client_num_samples)

      new_global_params = copy.deepcopy(global_params)
      for key in global_params.keys():
            new_global_params[key] = sum(local_params[i][key].float() * (client_num_samples[i] / total_samples) for i in range(len(client_samples)))

      model.load_state_dict(new_global_params)
      global_params = new_global_params
      if test:
        val_acc_fed, val_loss_fed=validate(model, test_loader, loss_fn)
      else:
        val_acc_fed, val_loss_fed=validate(model, val_loader, loss_fn)

      val_acc_fed_list.append(val_acc_fed)
      val_loss_fed_list.append(val_loss_fed)

      print(f'Iteration {t}-------------------')

  return val_acc_fed_list, val_loss_fed_list

In [ ]:
def train(epoch, model, inputs, targets, criterion, optimizer,J,client):
    model.train()

    inputs, targets = inputs.cuda(), targets.cuda()

    intermediate_output = model.get_intermediate_layers(inputs, n=1)
    features = torch.cat([x[:, 0] for x in intermediate_output], dim=-1)
    outputs = model.head(features)

    loss=criterion(outputs, targets)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch==J-1:
        running_loss = loss.item()
        _, predicted = outputs.max(1)
        total = targets.size(0)
        correct = predicted.eq(targets).sum().item()

        train_loss = running_loss
        train_accuracy = 100. * correct / total
        #print(f'Client: {client} Loss: {train_loss:.6f} Acc: {train_accuracy:.2f}%')

In [ ]:
def validate(model, val_loader, criterion):
    model.eval()
    val_loss = 0

    correct, total = 0, 0

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            inputs, targets = inputs.cuda(), targets.cuda()

            intermediate_output = model.get_intermediate_layers(inputs, n=1)
            features = torch.cat([x[:, 0] for x in intermediate_output], dim=-1)
            outputs = model.head(features)
            loss=criterion(outputs, targets)

            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    val_loss = val_loss / len(val_loader)
    val_accuracy = 100. * correct / total

    print(f'Validation Loss: {val_loss:.6f} Acc: {val_accuracy:.2f}%')
    return val_accuracy, val_loss

In [ ]:
#Vanilla FL with I.I.D sharding grid search
K = 100
C = 0.1
J = 4
T = 80
momentum = 0
loss_fn = nn.CrossEntropyLoss()

best_acc = 0
best_param=[0, 0, 0]

learning_rates = [1e-3, 1e-4]
weight_decay_values=[1e-3, 1e-4]

lr_counter=0
mt_counter=0
x_acc=torch.zeros(len(learning_rates),len(weight_decay_values),T)
x_loss=torch.zeros(len(learning_rates),len(weight_decay_values),T)
for lr in learning_rates:
    for weight_decay in weight_decay_values:
        model=copy.deepcopy(vits16_new)
        val_acc_fed_list, val_loss_fed_list=FedAvg(model, K, C, J, T, lr, momentum, weight_decay,loss_fn, client_loader, trainable_part='full', implementation='pyt',  client_loader_mask=client_loader_mask, test_loader=test_loader, test=False)
        val_acc_fed, val_loss_fed=validate(model, val_loader, loss_fn)

        if val_acc_fed > best_acc:
            best_acc = val_acc_fed
            best_param[0]=lr
            best_param[1]=weight_decay


        x_acc[lr_counter,mt_counter,:]=torch.tensor(val_acc_fed_list)
        x_loss[lr_counter,mt_counter,:]=torch.tensor(val_loss_fed_list)

        mt_counter+=1

        print("-"*30)
    mt_counter=0
    lr_counter+=1

print(f'Best validation accuracy: {best_acc:.2f}%')
print(f'Best learning rate: {best_param[0]}')
print(f'Weight decay: {best_param[1]}')

In [ ]:
colors=['green', 'orange']
labels_lr=['lr=1e-3', 'lr=1e-4']

ticks = [0] + [i for i in range(4, T, 5) if i != 0]
labels = [1] + [i + 1 for i in range(4, T, 5) if i != 0]

fig, (ax1, ax2) = plt.subplots( 1, 2,figsize=(10,5))
for i in range (0,2):
  ax1.plot(x_loss[i,0,:], label = labels_lr[i], color=colors[i])
ax1.set_xticks(ticks=ticks, labels=labels)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.grid(True)
ax1.set_title('Val_loss Weight decay=1e-3')
ax1.legend()
for i in range (0,2):
  ax2.plot(x_loss[i,1,:], label = labels_lr[i], color=colors[i])
ax2.set_xticks(ticks=ticks, labels=labels)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.grid(True)
ax2.set_title('Val_loss Weight decay=1e-4')
ax2.legend()

In [ ]:
#Test Vanilla FL with I.I.D sharding
lr = 1e-3
weight_decay=1e-4
T=200
vits16=copy.deepcopy(vits16_new)
test_acc_fed_list, test_loss_fed_list=FedAvg(vits16, K, C, J, T, lr, momentum, weight_decay,loss_fn, client_loader, trainable_part='full', implementation='pyt',  client_loader_mask=client_loader_mask, test_loader=test_loader,test=True)

In [ ]:
plt.figure(figsize=(16, 5))
plt.plot(test_loss_fed_list, label='Test Loss')
ticks = [0] + [i for i in range(4, T, 5) if i != 0]
labels = [1] + [i + 1 for i in range(4, T, 5) if i != 0]
plt.xticks(ticks=ticks, labels=labels)
plt.xlabel('Rounds')
plt.ylabel('Loss')
plt.grid(True)
plt.title('Test loss')
plt.legend()
plt.show()

In [ ]:
#non I.I.D. not balanced dataset definition
def create_dataset(N, K, num_class, train_indices, train_val):
  class_client = np.zeros((K,N), dtype=int)

  for i in range(K):
    class_client[i,:]=random.sample(range(num_class), N)

  unique_vals, nnzero_counts = np.unique(class_client, return_counts=True)

  unique_counts = np.zeros(num_class, dtype=int)
  for val, count in zip(unique_vals, nnzero_counts):
    unique_counts[val] = count

  class_to_indices = defaultdict(list)
  for idx in train_indices:
      label = train_val.targets[idx]
      class_to_indices[label].append(idx)

  for cls in class_to_indices:
      random.shuffle(class_to_indices[cls])

  client_indices_list = []
  client_dataset = []
  client_loader = []

  client_indices_list_mask = []
  client_dataset_mask = []
  client_loader_mask = []

  num_imm_per_class = int(len(train_indices)/(num_class))
  numel_per_classclient = []
  for cl in range(num_class):
    if unique_counts[cl]==0:
      numel_per_classclient.append(0)
    else:
      numel_per_classclient.append(int(num_imm_per_class/unique_counts[cl]))

  for client_id in range(K):
      class_client_list = class_client[client_id]
      indices_client = []
      indices_client_mask=[]

      for cls in class_client_list:
         indices_cls = class_to_indices[cls]
         sampled = indices_cls[0:numel_per_classclient[cls]]

         sampled_mask=indices_cls[0:max(1, int(0.15*numel_per_classclient[cls]))]

         class_to_indices[cls] = class_to_indices[cls][numel_per_classclient[cls]:]
         indices_client.extend(sampled)
         indices_client_mask.extend(sampled_mask)

      client_indices_list.append(indices_client)
      client_indices_list_mask.append(indices_client_mask)
      client_dataset.append(torch.utils.data.Subset(train_val, indices_client))
      client_dataset_mask.append(torch.utils.data.Subset(train_val, indices_client_mask))
      client_loader.append(torch.utils.data.DataLoader(client_dataset[-1], batch_size=32, shuffle=True))
      client_loader_mask.append(torch.utils.data.DataLoader(client_dataset_mask[-1], batch_size=1, shuffle=True))

  return client_indices_list,client_dataset,client_loader, client_loader_mask

In [ ]:
#non I.I.D balanced dataset definition
def create_balanced_dataset(N, K, num_class, train_indices, train_val):
   num_clients=K
   num_class_client=N
   mat_clients=np.full((num_clients,num_class_client), -1)
   toll=0
   nDisp=list(range(num_class))
   nUtil= np.zeros(num_class, dtype=int)

   for i in range(num_clients):
     for j in range(num_class_client):
        toll=random.choice(nDisp)
        mat_clients[i,j]=toll
        nDisp.remove(toll)
        if len(nDisp)==0:
          nDisp=list(range(num_class))

   values, counts = np.unique(mat_clients, return_counts=True)

   class_to_indices = defaultdict(list)
   for idx in train_indices:
      label = train_val.targets[idx]
      class_to_indices[label].append(idx)

   for cls in class_to_indices:
      random.shuffle(class_to_indices[cls])

   client_indices_list = []
   client_indices_list_mask = []
   client_dataset = []
   client_dataset_mask=[]
   client_loader = []
   client_loader_mask=[]

   numel_per_classclient = int(len(train_indices)/(num_class*N))

   for client_id in range(K):
      class_client = mat_clients[client_id]
      indices_client = []
      indices_client_mask=[]

      for cls in class_client:
         indices_cls = class_to_indices[cls]
         sampled = indices_cls[0:numel_per_classclient]

         sampled_mask=indices_cls[0:int(numel_per_classclient*0.15)]

         class_to_indices[cls] = class_to_indices[cls][numel_per_classclient:]
         indices_client.extend(sampled)

         indices_client_mask.extend(sampled_mask)

      client_indices_list.append(indices_client)
      client_dataset.append(torch.utils.data.Subset(train_val, indices_client))
      client_loader.append(torch.utils.data.DataLoader(client_dataset[-1], batch_size=32, shuffle=True))

      client_indices_list_mask.append(indices_client_mask)
      client_dataset_mask.append(torch.utils.data.Subset(train_val, indices_client_mask))
      client_loader_mask.append(torch.utils.data.DataLoader(client_dataset_mask[-1], batch_size=1, shuffle=True))

   return client_indices_list,client_dataset,client_loader, client_loader_mask

In [ ]:
#Control on the creation of dataset
def control(train_val,client_indices_list,K):
  for i in range(K):
    targetforclient=[]
    print(f"Client {i}:")
    for im in client_indices_list[i]:
      targetforclient.append(train_val.targets[im])
    print(Counter(targetforclient))
    print("-" * 40)

  all_indices = [idx for client_indices in client_indices_list for idx in client_indices]
  duplic = set(idx for idx in all_indices if all_indices.count(idx) > 1)
  if duplic:
    print(f"Number of duplicated images found: {duplic}")
  else:
    print("No duplicated images found.")

In [ ]:
#Creation of non I.I.D. dataset
Nc=[1,5,10,50]
num_class=int(len(set(targets)))
loss_fn = nn.CrossEntropyLoss()
K = 100
C = 0.1
J = [4,8,16]
T = 10
lr = 1e-3
momentum = 0
weight_decay = 1e-3

client_indices_list_noniid_balanced_list=[]
client_dataset_noniid_balanced_list=[]
client_loader_noniid_balanced_list=[]
client_loader_mask_noniid_balanced_list=[]

client_indices_list_noniid_list=[]
client_dataset_noniid_list=[]
client_loader_noniid_list=[]
client_loader_mask_noniid_list=[]

for N in Nc:
  client_indices_list_noniid_balanced,client_dataset_noniid_balanced,client_loader_noniid_balanced, client_loader_mask_noniid_balanced=create_balanced_dataset(N, K, num_class, train_indices, train_val)
  client_indices_list_noniid_balanced_list.append(client_indices_list_noniid_balanced)
  client_dataset_noniid_balanced_list.append(client_dataset_noniid_balanced)
  client_loader_noniid_balanced_list.append(client_loader_noniid_balanced)
  client_loader_mask_noniid_balanced_list.append(client_loader_mask_noniid_balanced)

  client_indices_list_noniid,client_dataset_noniid,client_loader_noniid, client_loader_mask_noniid=create_dataset(N,K, num_class,train_indices, train_val)
  client_indices_list_noniid_list.append(client_indices_list_noniid)
  client_dataset_noniid_list.append(client_dataset_noniid)
  client_loader_noniid_list.append(client_loader_noniid)
  client_loader_mask_noniid_list.append(client_loader_mask_noniid)

In [ ]:
#Vanilla FL with non I.I.D. balanced sharding for each J and Nc
T=[200,100,50]
lr = 1e-3
weight_decay = 1e-3
for n in range(4):
  for j in range(3):
    vits16_nn_iid_balanced=copy.deepcopy(vits16_new)
    FedAvg(vits16_nn_iid_balanced, K, C, J[j], T[j], lr, momentum, weight_decay,loss_fn, client_loader_noniid_balanced_list[n], trainable_part='full', implementation='pyt',  client_loader_mask=client_loader_mask_noniid_balanced_list[n], test_loader=test_loader, test=False)
    print(f'Validation for non i.i.d. balanced dataset with N={Nc[n]} and J={J[j]}')
    validate(vits16_nn_iid_balanced, val_loader, loss_fn)
    print("-"*50)

In [ ]:
#Vanilla FL with non I.I.D. unbalanced data fixing J=4, Nc=50
lr = 1e-3
weight_decay = 1e-3
vits16_nn_iid=copy.deepcopy(vits16_new)
FedAvg(vits16_nn_iid, K, C, 4, 200, lr, momentum, weight_decay,loss_fn, client_loader_noniid_list[3], trainable_part='full', implementation='pyt',  client_loader_mask=client_loader_mask_noniid_list[3], test_loader=test_loader, test=False)
print(f'Validation for non i.i.d. dataset with with N={50} and J={4}')
validate(vits16_nn_iid, val_loader, loss_fn)
print("-"*50)

In [ ]:
#Creation of the mask
def compute_fisher_mask(model, dataloader, sparsity, criterion):
  fisher_scores = {}
  prev_mask = {}

  model.eval()

  for param in model.head.parameters():
    param.requires_grad= False

  for param in model.parameters():
      if param.requires_grad:
          fisher_scores[param] = torch.zeros_like(param.data)
          prev_mask[param] = torch.ones_like(param.data)

  for round in range(5):
    for param in fisher_scores:
        fisher_scores[param].zero_()

    for inputs, targets in dataloader:
        inputs, targets = inputs.cuda(), targets.cuda()

        intermediate_output = model.get_intermediate_layers(inputs, n=1)
        features = torch.cat([x[:, 0] for x in intermediate_output], dim=-1)
        outputs = model.head(features)

        loss = criterion(outputs, targets)

        model.zero_grad()
        loss.backward()

        for param in model.parameters():
            if param.requires_grad and param.grad is not None:
              fisher_scores[param] += (param.grad.data.pow(2) * prev_mask[param])

    new_mask = {}
    all_scores = torch.cat([torch.flatten(v) for v in fisher_scores.values()])
    non_zero_scores=all_scores[all_scores!=0]
    k = int(sparsity[round] * non_zero_scores.numel())
    threshold, _ = torch.kthvalue(non_zero_scores, non_zero_scores.numel()-k)

    for param, score in fisher_scores.items():

        masked_score = score * prev_mask[param]
        current_mask = ((masked_score < threshold) * prev_mask[param]).float()
        new_mask[param] = current_mask
        prev_mask[param] = new_mask[param]

  return new_mask

In [ ]:
#Self implementation of SGDM with the addition of the mask
def sgdm_sparse (params, lr, momentum, dampening, weight_decay, nesterov, maximize,b, mask):
    for param in params:
        if param.grad is None:
            continue
        grad = param.grad.data

        if weight_decay!= 0:
          grad=grad+weight_decay*param.data

        if param not in b:
          b[param] = torch.zeros_like(param.data)

        if momentum!=0:
            b_toll = b[param]
            b_new = momentum * b_toll + (1 - dampening) * grad
            if nesterov:
               update=grad+momentum*b_new
            else:
              update=b_new
        else:
           update=grad
           b_new=0

        update = update * mask[param]

        if maximize:
          param.data=param.data+lr*update
          b[param] = b_new
        else:
          param.data=param.data-lr*update
          b[param] = b_new
    return b

In [ ]:
def train_sgd_sparse(epoch, model, inputs, targets, criterion, lr, momentum, weight_decay, mask):
     model.train()
     running_loss = 0.0
     correct = 0
     total = 0
     params=list(model.parameters())
     dampening=0
     nesterov=False
     maximize=False
     b={}

     inputs, targets = inputs.cuda(), targets.cuda()

     intermediate_output = model.get_intermediate_layers(inputs, n=1)
     features = torch.cat([x[:, 0] for x in intermediate_output], dim=-1)
     outputs = model.head(features)

     loss=criterion(outputs, targets)
     model.zero_grad()
     loss.backward()

     b=sgdm_sparse(params, lr, momentum, dampening, weight_decay, nesterov, maximize,b, mask)

In [ ]:
# Sparse FL with I.I.D. sharding
K = 100
C = 0.1
J = 4
momentum = 0
weight_decay = 1e-4
loss_fn = nn.CrossEntropyLoss()

lr = 1e-2
T=40
vits16_mask=copy.deepcopy(vits16_new)
FedAvg(vits16_mask, K, C, J, T, lr, momentum, weight_decay,loss_fn, client_loader, trainable_part='head', implementation='pyt', client_loader_mask=client_loader_mask, test_loader=test_loader, test=False)
lr = 5e-3
T=60
test_acc_fed_iid_list, test_loss_fed_iid_list=FedAvg(vits16_mask, K, C, J, T, lr, momentum, weight_decay,loss_fn, client_loader, trainable_part='full', implementation='self', client_loader_mask=client_loader_mask, test_loader=test_loader, test=True)

print(f'Validation for i.i.d dataset')
validate(vits16_mask, test_loader, loss_fn)
print("-"*50)

plt.figure(figsize=(16, 5))
plt.plot(test_loss_fed_iid_list, label='Test Loss')
ticks = [0] + [i for i in range(4, T, 5) if i != 0]
labels = [1] + [i + 1 for i in range(4, T, 5) if i != 0]
plt.xticks(ticks=ticks, labels=labels)
plt.xlabel('Rounds')
plt.ylabel('Loss')
plt.grid(True)
plt.title('Test loss')
plt.legend()
plt.show()

In [ ]:
# Sparse FL with non I.I.D. balanced sharding for each J and Nc
K = 100
C = 0.1
J = [4,8,16]
T=[60, 30, 15]
momentum = 0
weight_decay = 1e-3
loss_fn = nn.CrossEntropyLoss()
for n in range(4):
  for j in range(3):
    lr = 1e-2
    T_head=40
    vits16_mask_non_iid_bal=copy.deepcopy(vits16_new)
    FedAvg(vits16_mask_non_iid_bal, K, C, J[j], T_head, lr, momentum, weight_decay,loss_fn, client_loader_noniid_balanced_list[n], trainable_part='head', implementation='pyt', client_loader_mask=client_loader_mask_noniid_balanced_list[n], test_loader=test_loader, test=False)

    lr = 5e-3
    FedAvg(vits16_mask_non_iid_bal, K, C, J[j], T[j], lr, momentum, weight_decay,loss_fn, client_loader_noniid_balanced_list[n], trainable_part='full', implementation='self', client_loader_mask=client_loader_mask_noniid_balanced_list[n], test_loader=test_loader, test=False)

    print(f'Validation for non i.i.d.balanced dataset with with N={Nc[n]} and J={J[j]}')
    validate(vits16_mask_non_iid_bal, test_loader, loss_fn)
    print("-"*50)